## Scramble mapper test

<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle code on/off."></form>

In [13]:
from Bio import SeqIO
import pandas as pd
from importlib.machinery import SourceFileLoader
import sys
import os
import numpy as np
# add conda packages path
sys.path.append("/g/steinmetz/brooks/anaconda/envs/py35/lib/python3.5/site-packages/")
scrambleMap = SourceFileLoader("scrambleMap", "/g/steinmetz/brooks/git/steinmetz-lab/yeast2_0/scripts/scrambleMapper.py").load_module()

In [7]:
# important paths
ref = {
    "fa" : ('/g/steinmetz/project/IESY/genomes/annotations/scramble/genomes/S288C_reference_genome_R64-2-1_20150113/'
          'S288C_reference_sequence_R64-2-1_20150113.fsa'),
    "gff" : ('/g/steinmetz/project/IESY/genomes/annotations/scramble/genomes/S288C_reference_genome_R64-2-1_20150113/'
          'saccharomyces_cerevisiae_R64-2-1_20150113_annotation_only.gff')}

query = {
    "fa" : ('/g/steinmetz/project/IESY/genomes/annotations/scramble/genomes/'
            'JS710.fa'),
    "gff" : ('/g/steinmetz/project/IESY/genomes/annotations/scramble/genomes/'
            'test.gff')
}

In [8]:
df = scrambleMap.readGFF(ref['gff'])
seq = scrambleMap.readFASTA_SeqIO(ref["fa"])
#df["feature"].unique()
#thisentry3 = df.loc[(df.index.get_level_values('seqname') == "chrIX") & (df.index.get_level_values('start') > 173365)]
#thisentry = thisentry[thisentry['attribute'].str.contains('DAL4')]
thisentry = df
annotations2keep = [
        'gene',
        'ncRNA_gene',
        'tRNA_gene',
        'centromere',
        'rRNA_gene'
        ]
#annotations2keep = ["tRNA_gene",'rRNA_gene']
thisentry = thisentry[thisentry['feature'].isin(annotations2keep)]

# thisentry = df.loc[(df.index.get_level_values('seqname') == "chrII")]
#thisentry = thisentry[thisentry['attribute'].str.contains('YIR017C|YIR018W')]
#thisentry = thisentry[thisentry['attribute'].str.contains('YIR017C')]
#thisentry2 = thisentry[thisentry['attribute'].str.contains('tT%28AGU%29I1')]
thisentry3 = thisentry[thisentry['attribute'].str.contains('YIR034C')]

#out = scrambleMap.mapGFF(ref['fa'],thisentry.iloc[[0]], query['fa'])
#out = scrambleMap.mapMultipleGFF(ref['fa'],thisentry, query['fa'])
#out2= scrambleMap.mapMultipleGFF(ref['fa'],thisentry2, query['fa'])
#out3= scrambleMap.mapMultipleGFF(ref['fa'],thisentry3, query['fa'])
# scrambleMap.writeGFF(out,"JS710.gff")

In [9]:
to_fa = query["fa"]
seg_file = "/g/steinmetz/brooks/git/steinmetz-lab/yeast2_0/scramble/SynIXR_segmentTable.txt"
nproc=32

In [10]:
tmp = scrambleMap.addSeg(to_fa,seg_file)

TypeError: unhashable type: 'list'

In [ ]:
out = []
out.append("a")

In [ ]:
out

In [ ]:
from Bio import SeqIO
import os
import tempfile
import uuid
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from itertools import repeat
if os.path.isfile(seg_file):
    with open(seg_file,'r') as f:
        df = pd.DataFrame.from_csv(f,sep='\t')
    temp_file = tempfile.TemporaryFile(mode='w')
    temp_file = str(uuid.uuid4())
    with open(temp_file,'w') as f2:
        for i in df.index.get_level_values("number"):
            f2.write(">"+str(i) + "\n")
            f2.write(str(df.loc[i].seq)+"\n")
    dummy_gff = pd.DataFrame.from_records([{
        "seqname":"dummy",
        "start":1,
        "end":1,
        "source":"ANB",
        "feature":"engineered_region",
        "score":".",
        "strand":"+",
        "frame":".",
        "attribute":"ID=;"
    }],index=("seqname","start","end"))
    thisfrom = scrambleMap.readFASTA_SeqIO(temp_file)
    #out = pd.DataFrame()
    results = []
    segments = [x for x in thisfrom.keys()]
    gff_arg = dummy_gff
    for i in range(0,len(segments)):
        ii = segments[i]
        if i == 0:
            gff_arg.iloc[i].attribute = "ID=" + str(ii) + ";"
        else:
            gff_arg = gff_arg.append(dummy_gff)
            gff_arg.iloc[i].attribute = "ID=" + str(ii) + ";"
    gff_arg = [gff_arg.iloc[[x]] for x in range(0,gff_arg.shape[0])]
    pool = mp.Pool(processes=nproc)
    #gff_arg = [gff_in.iloc[[x]] for x in range(0,gff_in.shape[0])]
    results = pool.starmap(scrambleMap.mapGFF_direct, zip(gff_arg,repeat(to_fa),[thisfrom[s] for s in segments]))
    out = pd.concat(results)